In [29]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import pickle


In [30]:
import pickle
## Load 
model = load_model('model.h5')

with open('one_hot_encoder_geography.pkl', "rb") as file:
    label_encoder_geo=pickle.load(file)
    
with open('label_encoder_gender.pkl','rb') as file:
    label_encode_gender = pickle.load(file)
    
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [31]:
new_input ={
    'CreditScore': 720,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 35,
    'Tenure': 5,
    'Balance': 125000.0,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 65000
}

In [37]:
# One-hot Encode 'Geography'
geo_encoded = label_encoder_geo.transform([[new_input['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\mohsi\OneDrive\Desktop\deepLearning\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [38]:
input_df=pd.DataFrame([new_input])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,720,France,Male,35,5,125000.0,2,1,1,65000


In [39]:
# Label Encode 'Gender'
input_df['Gender'] = label_encode_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,720,France,1,35,5,125000.0,2,1,1,65000


In [40]:
# Drop original Geography and concatenate encoded columns
input_data = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,720,1,35,5,125000.0,2,1,1,65000,1.0,0.0,0.0


In [42]:
input_scaled = scaler.transform(input_data)
input_scaled

array([[ 0.71691912,  0.90750738, -0.37493284, -0.00573947,  0.78042101,
         0.80883036,  0.64104192,  0.97067965, -0.60578899,  1.00175153,
        -0.57831252, -0.57773517]])

In [ ]:
# Make prediction
probability = model.predict(input_scaled)[0][0]

# Interpret prediction
churned = int(probability > 0.5)  # 1 = churn, 0 = stay

# Output the result
if churned:
    print(f"🔴 Prediction: The customer is likely to churn. (Probability: {probability:.2f})")
else:
    print(f"🟢 Prediction: The customer is likely to stay. (Probability: {probability:.2f})")

1/1 [==============================] - 0s 67ms/step
🟢 Prediction: The customer is likely to stay.
Confidence: 97.15%
